In [1]:
import json
import os

!export export CUDA_HOME="/usr/local/cuda-11.1" 
os.environ['CUDA_HOME'] = '/usr/local/cuda-11.1'

In [2]:
with open('datasets/ActionGenome/anno_frames_bicls/custom_prediction.json') as f:
    data_graphs = json.load(f)


In [3]:
with open('datasets/ActionGenome/anno_frames_bicls/custom_data_info.json') as f:
    data_info = json.load(f)

In [5]:
with open('datasets/ActionGenome/anno_frames_bicls/scene_validation_annotations.json') as f:
    val_info = json.load(f)

In [6]:
data_graphs['0'].keys()

dict_keys(['bbox', 'bbox_labels', 'bbox_scores', 'rel_pairs', 'rel_labels', 'rel_scores', 'rel_all_scores'])

In [7]:
reformat_data_info = {}
for item in data_info['idx_to_files']:
    reformat_data_info[item.split('/')[-1]] = -1

In [8]:
for item in val_info:
    reformat_data_info[item['image_id']] = item['label_id']

In [9]:
graph_labels = list(reformat_data_info.values())

In [10]:
def change_format(selected_image,box_topk = 15,rel_topk = 15):    
    all_rel_labels = selected_image['rel_labels']
    all_rel_scores = selected_image['rel_scores']
    all_rel_pairs = selected_image['rel_pairs']
    edges = []
    for i in range(len(all_rel_pairs)):
        
        if all_rel_pairs[i][0] < box_topk and all_rel_pairs[i][1] < box_topk:
            source = all_rel_pairs[i][0]
            target = all_rel_pairs[i][1]
            score = all_rel_scores[i]
            edges.append({'source':source, 'target':target, 'relation': all_rel_labels[i], 'score': score})

    edges = edges[:rel_topk]
    return edges

In [11]:
graph_list = []
slice_stop = 10000
for i in range(399):
    if i == slice_stop:
        break
    edges = change_format(data_graphs[str(i)],box_topk = 100,rel_topk = 100)
    graph_list.append(edges)

In [12]:
import torch
from torch_geometric.data import Data, DataLoader

In [13]:
def process_graph(graph_representation, label):
    edge_sources = [edge['source'] for edge in graph_representation]
    edge_targets = [edge['target'] for edge in graph_representation]
    edge_relations = [edge['relation'] for edge in graph_representation]
    edge_scores = [edge['score'] for edge in graph_representation]

    all_nodes = edge_sources + edge_targets
    all_nodes= list(set(all_nodes))
 
    node_features_neig = [edge_sources.count(node) for node in all_nodes]
    node_features_names = all_nodes
    node_features = [[name,neig] for name,neig in zip(node_features_names,node_features_neig)]
    edge_sources_new_index = [all_nodes.index(node) for node in edge_sources]
    edge_edge_targets_new_index = [all_nodes.index(node) for node in edge_targets]  

    x = torch.tensor(node_features, dtype=torch.float)
    edge_index = torch.tensor([edge_sources_new_index, edge_edge_targets_new_index], dtype=torch.long)
    edge_attr = torch.tensor((node_features), dtype=torch.float)
    
    return Data(x=x, edge_index=edge_index,num_nodes = len(all_nodes), edge_attr=edge_attr, y=torch.tensor([label], dtype=torch.long))


In [14]:
data_list = [process_graph(graph, label) for graph, label in zip(graph_list, graph_labels)]


In [15]:
torch.save(data_list, './outputs/custom_train/data_list.pth')

In [16]:
loader = DataLoader(data_list, batch_size=32, shuffle=True)
for batch in loader:
    # Your training or processing code here
    print(batch)

Batch(x=[1401, 2], edge_index=[2, 3200], edge_attr=[1401, 2], y=[32], num_nodes=1401, batch=[1401], ptr=[33])
Batch(x=[1493, 2], edge_index=[2, 3200], edge_attr=[1493, 2], y=[32], num_nodes=1493, batch=[1493], ptr=[33])
Batch(x=[1415, 2], edge_index=[2, 3200], edge_attr=[1415, 2], y=[32], num_nodes=1415, batch=[1415], ptr=[33])
Batch(x=[1461, 2], edge_index=[2, 3200], edge_attr=[1461, 2], y=[32], num_nodes=1461, batch=[1461], ptr=[33])
Batch(x=[1439, 2], edge_index=[2, 3200], edge_attr=[1439, 2], y=[32], num_nodes=1439, batch=[1439], ptr=[33])
Batch(x=[1533, 2], edge_index=[2, 3200], edge_attr=[1533, 2], y=[32], num_nodes=1533, batch=[1533], ptr=[33])
Batch(x=[1499, 2], edge_index=[2, 3200], edge_attr=[1499, 2], y=[32], num_nodes=1499, batch=[1499], ptr=[33])
Batch(x=[1416, 2], edge_index=[2, 3200], edge_attr=[1416, 2], y=[32], num_nodes=1416, batch=[1416], ptr=[33])
Batch(x=[1452, 2], edge_index=[2, 3200], edge_attr=[1452, 2], y=[32], num_nodes=1452, batch=[1452], ptr=[33])
Batch(x=[1

/home/hronis10/miniconda3/envs/sgg-classification/lib/python3.7/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [17]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv, global_mean_pool
import random

In [18]:
class GraphSAGEBinaryClassification(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super(GraphSAGEBinaryClassification, self).__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
        self.conv3 = SAGEConv(hidden_channels, hidden_channels)
        self.conv4 = SAGEConv(hidden_channels, hidden_channels)
        self.lin = torch.nn.Linear(hidden_channels, 1)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv4(x, edge_index)
        x = F.relu(x)
        
        x = global_mean_pool(x, batch)
        return self.lin(x).squeeze()  # Squeeze for binary classification


In [19]:
# Shuffle the data_list
random.shuffle(data_list)

# Determine the split indices
train_size = int(0.8 * len(data_list))
test_size = len(data_list) - train_size

train_dataset = data_list[:train_size]
test_dataset = data_list[train_size:train_size+test_size]

train_loader = DataLoader(train_dataset, batch_size=60, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=60, shuffle=False)

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GraphSAGEBinaryClassification(2, 64).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
criterion = torch.nn.BCEWithLogitsLoss()

In [21]:
model.train()
for epoch in range(10000):
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        
        loss = criterion(out, data.y.float())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}")

Epoch 1, Loss: 0.8411525289217631
Epoch 2, Loss: 0.7687631547451019
Epoch 3, Loss: 0.7473030984401703
Epoch 4, Loss: 0.6976941625277201
Epoch 5, Loss: 0.6871119836966196
Epoch 6, Loss: 0.6843150953451792
Epoch 7, Loss: 0.6974554558595022
Epoch 8, Loss: 0.6955693960189819
Epoch 9, Loss: 0.6855224271615347
Epoch 10, Loss: 0.6779772937297821
Epoch 11, Loss: 0.6822356780370077
Epoch 12, Loss: 0.6816608309745789
Epoch 13, Loss: 0.7009138961633047
Epoch 14, Loss: 0.6901917556921641
Epoch 15, Loss: 0.6870162785053253
Epoch 16, Loss: 0.690274586280187
Epoch 17, Loss: 0.6710820893446604
Epoch 18, Loss: 0.6747739116350809
Epoch 19, Loss: 0.6719612379868826
Epoch 20, Loss: 0.669402003288269
Epoch 21, Loss: 0.69807897011439
Epoch 22, Loss: 0.6927901605765024
Epoch 23, Loss: 0.6812406480312347
Epoch 24, Loss: 0.683365136384964
Epoch 25, Loss: 0.6869465907414755
Epoch 26, Loss: 0.6915593345959982
Epoch 27, Loss: 0.6590472559134165
Epoch 28, Loss: 0.6810129682223002
Epoch 29, Loss: 0.6843675275643667

In [22]:
# Evaluation
model.eval()

correct = 0
total = 0

with torch.no_grad():
    for data in test_loader:
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)
        predicted = (torch.sigmoid(out) > 0.5).long()
        total += data.y.size(0)
        correct += (predicted == data.y).sum().item()

print('Accuracy: {:.4f}'.format(correct / total))

Accuracy: 0.5250


: 